In [41]:
#Import
from spotify import Client, OAuth
import pandas as pd
import pygn
import pickle

In [42]:
#Grace Note import
clientID = '740529234-9D358964C751528ED8F5DA9B0D060C84' # Enter your Client ID here
userID = pygn.register(clientID)

In [43]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [44]:
spotifyClient = Client(auth)

In [45]:
spotifyAPI = spotifyClient.api

In [46]:
def getTrackAttributes(trackIdList = []):
    print(trackIdList)
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = filter(None, trackList)
    trackList = [x for x in trackList if x is not None]
    return trackList

In [47]:
def getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [48]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    #tracks = list(filter(None, tracks))
    return tracks


In [49]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        if trackObj['id'] != None:
            flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [50]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)

    # Get list of track ids
    trackIds = list(plTracksDF['id'])

    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)

    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)

    # Merging 2 data frames
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')

    # Droping duplicate ID columns
    resultDF = resultDF.T.drop_duplicates().T

    return resultDF
    
    

In [ ]:
res = pipeLineFunc('2kpoUUJ5a4Cw3feTkFJhZ2','1276640268','top100')
res

In [51]:
playListData = [{'user':'11124786922','playlist_id':'19zHVSpAeY9zVXBML8gCpS','genre':'Pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX5J7FIl4q56G','genre':'Metal'},
{'user':'spotify','playlist_id':'37i9dQZF1DX0XUsuxWHRQd','genre':'Rap'},
{'user':'spotify','playlist_id':'37i9dQZF1DX1lVhptIYRda','genre':'Country'},
{'user':'spotify','playlist_id':'37i9dQZF1DXaImRpG7HXqp','genre':'Acoustic'},
{'user':'dinhcn','playlist_id':'0gHKn2g9fC0Dur6xFx0TGI','genre':'EDM'}]

In [52]:
playlistDF = pd.DataFrame(playListData)
print(playlistDF)

      genre             playlist_id         user
0       Pop  19zHVSpAeY9zVXBML8gCpS  11124786922
1     Metal  37i9dQZF1DX5J7FIl4q56G      spotify
2       Rap  37i9dQZF1DX0XUsuxWHRQd      spotify
3   Country  37i9dQZF1DX1lVhptIYRda      spotify
4  Acoustic  37i9dQZF1DXaImRpG7HXqp      spotify
5       EDM  0gHKn2g9fC0Dur6xFx0TGI       dinhcn


In [53]:
trackDFMain = pd.DataFrame()
for i in range(len(playlistDF)):
    resDF = pipeLineFunc(playlistDF["playlist_id"][i],playlistDF["user"][i],playlistDF["genre"][i])
    trackDFMain = trackDFMain.append(resDF)

['2wYHz0GOHV49Xezd5mWTDP', '5SxkdsY1ufZzoq9iXceLw9', '2ARqIya5NAuvFVHSN3bL0m', '0BlY60NrN0fFWbdW3RW40q', '7iDa6hUg2VgEL1o1HjmfBn', '3Vo4wInECJQuz9BIBMOu8i', '4ofvoCdXq99vpMZO4Cfp8x', '6T8cJz5lAqGer9GUHGyelE', '3YU6vJbjYUG0tiJyXf9x5V', '6mrKP2jyIQmM0rw6fQryjr', '2ekn2ttSfGqwhhate0LSR0', '0tgVpDi06FyKpA1z0VMD4v', '4IWAyPf1KMq7JCyGeCjTeH', '1Qg9jWGDkKAGZXj5v2OG59', '6tHWl8ows5JOZq9Yfaqn3M', '08bNPGLD8AhKpnnERrAc6G', '79jX8RM5CgPqDKdGEKNW9K', '1LhMopPAallLeaeNutqbgS', '7fCNUWi6uflDTQ08srxMZk', '1wjzFQodRWrPcQ0AnYnvQ9', '12SJpYZEdj4kzWZ8FPFg4Y', '5Htb1uFQ1KrkFXlefS8oGj', '7wrDRQgHlvDnimrRHfQZxt', '5BG3hnp2SmFmoBIs3YTqsm', '2Pvr8rYuh5J8V7QglJhH6W', '7CneiDesUKFJO3Ix01dnfr', '4kVo85bcmBkvMBcJl42XCA', '3tbNT2X5tfMIVeUflYbZUZ', '7anBA0IdgFaN3Pbyk2tog0', '3E5NUvTLszM8uymKQ227MP', '54j7EaJPDmSZYcNYvLSJ78', '7ef4DlsgrMEH11cDZd32M6', '76S09sHj7GGMJAtOLjmVQ2', '6zOhgKfbMiQWToE6K13s2s', '3pIigjAWkzwAonXpJ1fmSF', '3EypXSP7JVjtusvnBtAIBo', '2tek8UnoiJdkmpPMqa5KRQ', '45Egmo7icyopuzJN0oMEdk', '7bXvOMQ3Ur

['1KDX8TC34n6CNryBEEMclR', '6aWiGv6hPG0o3ri7QHNs8t', '1aS1uzcAC14lduwuNJZjqE', '0Q3SC6kEhxYagDP3bFe5K9', '4WCNiW7DJFE6h94q5NPZmZ', '59ZSmCCe6Ao1qtptCqX9Tk', '1xQv4unOladJFtLUOy2eb7', '2U0pVx4m1Kdm1Gsjjm6iq8', '1i1fxkWeaMmKEB4T7zqbzK', '12MqM3S0fTvR6X4Jt1Xvn6', '33lXq3csEWwYNn6x0sM3Et', '05dL7chWJaVjMJnGO1YCKk', '22qUyTlEvJf6DAlyQjMADe', '5jnp8E61wcoF6qIEtDHQnJ', '2sNvitW3TxiTeC9xT9f2ZZ', '1LeItUMezKA1HdCHxYICed', '2Mh0jU5IyRTQfpyMnvki5Z', '0wRaAh1Km76Blq4qMJjjvN', '46lFttIf5hnUZMGvjK0Wxo', '27NV2KxoQ8WuLMqlTDI61F', '2bKhIGdMdcqCqQ2ZhSv5nE', '6hJSH3jS9jG6khVclppBJw', '2LGfcxG9LcDOxnltIPE1Zq', '4f13AaRyukUTWleyKXaRDh', '2RpKh7kXSdO8NLrW9VQ46p', '6m3ZWIXhjoV76syT1j2oE9', '0NWQTyapmz4GuDTSN9xTB7', '69mlHIPayAZVzC32RaVty5', '5cr2MqXUs0TGaDByQnKVnX', '2H8LPaY3NSjmOFMH2Rygnz', '2aLIBIlebwNpljSxFczUcL', '7gBj0VgcuAgkXkiRRYvSmK', '7sG2bWi2eDtvX9vX8yC1Gn', '2kYqdSlrtovVMMIn6ykzba', '2qJLp4SATqV1OP5IDN7SMl', '6RXdg3NwyLU5pHUwkZq6VA', '3xQjivjUZ4KnUslduTEYKC', '1KcnJTiXJXAqW5bB7zpkBC', '1hZk9dktp4

In [54]:
trackDFMain

,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Falling,Lina Mayer,Pop,2wYHz0GOHV49Xezd5mWTDP,Falling,0.0315,0.447,201385,0.594,1.02e-06,5,0.111,-5.996,1,0.0448,164.05,4,0.521
1,No Tears Left To Cry,Ariana Grande,Pop,5SxkdsY1ufZzoq9iXceLw9,No Tears Left To Cry,0.0375,0.703,205947,0.696,5.65e-06,0,0.274,-5.482,1,0.0529,121.969,4,0.366
2,The Middle,Zedd,Pop,2ARqIya5NAuvFVHSN3bL0m,The Middle,0.208,0.748,184732,0.652,0,7,0.128,-3.02,1,0.058,106.914,4,0.431
3,Never Be the Same (Radio Edit),Camila Cabello,Pop,0BlY60NrN0fFWbdW3RW40q,Never Be the Same - Radio Edit,0.156,0.652,227077,0.682,0.00013,0,0.121,-4.38,1,0.0788,129.942,4,0.269
4,All Your Fault: Pt. 2,Bebe Rexha,Pop,7iDa6hUg2VgEL1o1HjmfBn,Meant to Be (feat. Florida Georgia Line),0.047,0.643,163870,0.783,0,10,0.083,-6.458,1,0.0856,154.084,4,0.579
5,Finesse (Remix) [feat. Cardi B],Bruno Mars,Pop,3Vo4wInECJQuz9BIBMOu8i,Finesse (Remix) [feat. Cardi B],0.0185,0.704,217289,0.859,0,5,0.0215,-4.877,0,0.0996,105.115,4,0.926
6,Pray For Me (with Kendrick Lamar),The Weeknd,Pop,4ofvoCdXq99vpMZO4Cfp8x,Pray For Me (with Kendrick Lamar),0.0805,0.728,211440,0.668,1.49e-05,2,0.102,-5.02,1,0.126,100.537,4,0.162
7,Scary Hours,Drake,Pop,6T8cJz5lAqGer9GUHGyelE,God's Plan,0.0283,0.755,198960,0.448,3.28e-05,7,0.514,-9.587,1,0.107,77.179,4,0.318
8,Him & I,G-Eazy,Pop,3YU6vJbjYUG0tiJyXf9x5V,Him & I,0.0534,0.589,268867,0.731,0,2,0.308,-6.343,1,0.0868,87.908,4,0.191
9,Perception,NF,Pop,6mrKP2jyIQmM0rw6fQryjr,Let You Down,0.33,0.659,212120,0.716,0,5,0.173,-5.673,0,0.111,148.015,4,0.437


In [55]:
trackDFMain.to_csv('1000Tracks.csv')

In [56]:
uniquetrackDF = trackDFMain.drop_duplicates('id')

In [57]:
uniquetrackDF.to_csv('1000TracksUnique.csv')

In [58]:
uniquetrackDF = pd.read_csv('1000TracksUnique.csv')

In [59]:
uniquetrackDF['artist'][0]

'Lina Mayer'

In [60]:
def getGNMood(art,alb,trk,trk_id):

    # art = str(inpdf['artist'])
    # alb =  str(inpdf['album'])
    # trk =  str(inpdf['track_name'])
    #print(art, alb, trk)
    mood_data = ""
    try:
        moodData = pygn.search(clientID=clientID,
                               userID=userID,
                               artist=art, 
                               album=alb,
                               track=trk)
        #print(moodData)
        
        if moodData['mood'] != {} or moodData['mood'] != None:
            if '1' in moodData['mood'].keys():
                if 'TEXT' in moodData['mood']['1'].keys():
                    mood_data = moodData['mood']['1']['TEXT']
        op = {'track_id':trk_id, 'mood':mood_data}
        return op
    except:
        print("error")
        op = {'track_id':trk_id, 'mood':mood_data}
        return op

In [61]:
def getAllMoodData(inpdf):
    opList = []
    for i in range(len(inpdf)):
        #print(inpdf.iloc[i,:])
        #print(inpdf['artist'][i])

        art = str(inpdf['artist'][i])
        alb =  str(inpdf['album'][i])
        trk =  str(inpdf['track_name'][i])
        trk_id =  str(inpdf['id'][i])
        print(i,art,alb,trk,trk_id)
        op = getGNMood(art,alb,trk,trk_id)
        opList.append(op)
    return opList

In [63]:
moodList = []


In [64]:

mood = getAllMoodData(uniquetrackDF)
moodList = moodList + mood



0 Lina Mayer Falling Falling 2wYHz0GOHV49Xezd5mWTDP
1 Ariana Grande No Tears Left To Cry No Tears Left To Cry 5SxkdsY1ufZzoq9iXceLw9
2 Zedd The Middle The Middle 2ARqIya5NAuvFVHSN3bL0m
3 Camila Cabello Never Be the Same (Radio Edit) Never Be the Same - Radio Edit 0BlY60NrN0fFWbdW3RW40q
4 Bebe Rexha All Your Fault: Pt. 2 Meant to Be (feat. Florida Georgia Line) 7iDa6hUg2VgEL1o1HjmfBn
5 Bruno Mars Finesse (Remix) [feat. Cardi B] Finesse (Remix) [feat. Cardi B] 3Vo4wInECJQuz9BIBMOu8i
6 The Weeknd Pray For Me (with Kendrick Lamar) Pray For Me (with Kendrick Lamar) 4ofvoCdXq99vpMZO4Cfp8x
7 Drake Scary Hours God's Plan 6T8cJz5lAqGer9GUHGyelE
8 G-Eazy Him & I Him & I 3YU6vJbjYUG0tiJyXf9x5V
9 NF Perception Let You Down 6mrKP2jyIQmM0rw6fQryjr
10 Dua Lipa Dua Lipa (Deluxe) New Rules 2ekn2ttSfGqwhhate0LSR0
11 Ed Sheeran ÷ (Deluxe) Perfect 0tgVpDi06FyKpA1z0VMD4v
12 Imagine Dragons Evolve Whatever It Takes 4IWAyPf1KMq7JCyGeCjTeH
13 Maroon 5 Red Pill Blues (Deluxe) Wait 1Qg9jWGDkKAGZXj5v2OG59
14 Baz

114 Pallbearer Dropout Dropout 1PaejblOMlnQiMUUn85DBF
115 Zeal & Ardor Waste Waste 3nPjOr3fbBHzsHNCWnEBC7
116 Mos Generator Shadowlands Shadowlands 7lxPjFRAPmlVkHD6ded89T
117 Wolf King Loyal to the Soil Hail the Ash 5EqVCH2hex0pQHiO0FH9bX
118 Marduk Werwolf Werwolf 7IoJyfqqHtm6Sq6PPMKilD
119 At The Gates Daggers of Black Haze Daggers of Black Haze 57n95eCpLXKEc6ww9Q2PKR
120 Aura Noir Shades Ablaze Shades Ablaze 77CdUFrya4BKSHyNQRjsmA
121 Siamese Party Monster Party Monster 7iV1VkQhsPrI2h9mPQYDfm
122 Nonexist A Meditation Upon Death A Meditation Upon Death 0rw4O9dcTXWYNygvsBHnab
123 Nekrokraft Lechery Lechery 4Tkz6tlWzQckC7Prr4Q0m6
124 Light This City A Grotesque Reflection A Grotesque Reflection 750WF02D3eoxYwCesfsweY
125 Blood Tsunami Grave Condition Allegory of the Cave 6rii5E2Smr3EUf3an3vrAg
126 The Agony Scene Hand of the Divine Hand of the Divine 3A6czTc03gZxTDMVNEVjpa
127 Orange Goblin Sons Of Salem Sons Of Salem 2jHJZPMCFUVaYnQBOUiNcU
128 Alkaloid Chaos Theory and Practice Chaos

230 Luke Combs This One's for You One Number Away 4gB7HrYHbJVJ5RFOjxmoq4
231 Dylan Scott Hooked Hooked 6LF5IVPG7xBcvPgI6Bfcsl
232 Sugarland Babe Babe 5FNeJveCmVB2TPDbIYqK0M
233 Sir Rosevelt Something 'Bout You Something 'Bout You 6QkObBYNxVVUYt4FSpnTqY
234 Jake Owen Jake Owen I Was Jack (You Were Diane) 52K3RIVDetSuZPzcTmIVoY
235 Thomas Rhett Life Changes Leave Right Now 0wyja8xD9cNhDtfuY0dVMP
236 Camila Cabello Never Be the Same (feat. Kane Brown) Never Be the Same (feat. Kane Brown) 0xk4woi8ENN8VWFDZTcF8h
237 Morgan Wallen Up Down (Feat. Florida Georgia Line) Up Down (Feat. Florida Georgia Line) 3LhzpgimYSZhTx5KuchwVX
238 High Valley Dear Life She's With Me 2p2cCrOaNrIOLk5ArtlHy6
239 Dierks Bentley Woman, Amen Woman, Amen 4YX7OBp0yoMwC6eZNNg5ky
240 Cole Swindell Break Up in the End Break Up in the End 7uVkYr8gxcPEgqohvynpdN
241 Brandon Lay Speakers Speakers, Bleachers And Preachers 0Is6MRHWKe7y2xIxDkvO6l
242 Jason Aldean Rearview Town Rearview Town 3yHTTHbT058Ow0favhZO38
243 Taylor S

327 Ramin Djawadi Westworld: Season 1 (Selections from the HBO® Series) Black Hole Sun 7Ap7h823EAc1GBj6JKHBEK
328 Acoustic Guitar Tribute Players Acoustic Guitar Plays New Pop Music: She Will Be Loved Hey, Soul Sister 0oqAf7pxrRFIj9hlebCRAQ
329 David Garrett Music Cry Me A River 4qEPcXwif4thULut32oPqK
330 Instrumental Pop Songs You Found Me - Instrumental Pop Songs Use Somebody 6Ku0bY5NhoSq9kU1fYz8BH
331 Ely Jaffe On the Voice of Guitar, Vol. II Skyfall 31J06ok5AORJFFF3xDVste
332 Alison Sparrow All Of Me All Of Me Violin Cover 2vcvckOg239y6j18Ei3JV0
333 Davy Graham 3/4 AD Angi 2wxpgRChMOPsE9lebeEzMj
334 The O'Neill Brothers Group Current Pop Hits: Instrumental Versions Stay (Instrumental Version) 7MfTO5QDPGGWD846Ar0Sva
335 Gavin Mikhail Spectrum (Say My Name) (Florence + The Machine Cover) The A Team (Angels To Fly) (Instrumental) 5EufMaUYHnFgnNE3C2wLd0
336 David Garrett Rock Symphonies Smells Like Teen Spirit 3FT92lQ7y5LzT2QOlhuwTg
337 Bill Frisell All We Are Saying... Beautiful Boy 6

435 DJ Khaled Major Key Do You Mind 4VJ7mnW4wjjIgPBM6KWvPu
436 Jack Ü Skrillex and Diplo present Jack Ü Where Are Ü Now (with Justin Bieber) 66hayvUbTotekKU3H4ta1f
437 Jack Ü Skrillex and Diplo present Jack Ü To Ü (feat. AlunaGeorge) 3UgSQu6WwrXfKKDq019IHE
438 Porter Robinson Shelter Shelter 3ISYGHoYnu4gos6JNMfL4z
439 Marshmello Joytime Summer 1Hk0QRlUFCHYG6zIvUh0Xd
440 Gallant Weight in Gold (The Remixes) Weight in Gold - Ekali Remix 4mRpWePw7B8ePqfP7aMD7S
441 The Chainsmokers All We Know All We Know 2rizacJSyD9S1IQUxUxnsK
442 Sound Remedy Ashes Spirals (feat. King Deco) 6XtSFFAy65MocQiaHMj2bp
443 Calvin Harris My Way My Way 1vvNmPOiUuyCbgWmtc6yfm
444 Madeon Adventure (Deluxe) You're On 4PTPZeJlK1rYlYr6bf11hk
445 Madeon Adventure (Deluxe) Pay No Mind 3rsq3qtqNBId48c0IVraTS
446 Cash Cash Blood, Sweat & 3 Years Hero (feat. Christina Perri) 0jPU39bL0SrCmYc1RwbFkX
447 Tritonal Broken Broken 3NpncOG0HgkKbMAF8DTCma


In [65]:
moodList_BK  = moodList

In [66]:
a = {"mood1000":moodList}
with open('mood1000.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [67]:
moodListMain = moodList

In [68]:
# merge
moodDF = pd.DataFrame(moodListMain)

In [69]:
moodDF

,mood,track_id
0,Sensual,2wYHz0GOHV49Xezd5mWTDP
1,,5SxkdsY1ufZzoq9iXceLw9
2,,2ARqIya5NAuvFVHSN3bL0m
3,Yearning,0BlY60NrN0fFWbdW3RW40q
4,Sensual,7iDa6hUg2VgEL1o1HjmfBn
5,Cool,3Vo4wInECJQuz9BIBMOu8i
6,Cool,4ofvoCdXq99vpMZO4Cfp8x
7,,6T8cJz5lAqGer9GUHGyelE
8,Cool,3YU6vJbjYUG0tiJyXf9x5V
9,Rowdy,6mrKP2jyIQmM0rw6fQryjr


In [71]:
mainDF = pd.merge(uniquetrackDF, moodDF, how='left', left_on=['id'], right_on=['track_id'])

In [72]:
mainDF = mainDF.drop(['id'],axis=1)

In [73]:
mainDF.to_csv('dataWithLabel1000.csv')

In [84]:
mainDF

,Unnamed: 0,album,artist,genre,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,mood,track_id
0,0,Falling,Lina Mayer,Pop,Falling,0.031500,0.447,201385,0.594,0.000001,5,0.1110,-5.996,1,0.0448,164.050,4,0.5210,Sensual,2wYHz0GOHV49Xezd5mWTDP
1,1,No Tears Left To Cry,Ariana Grande,Pop,No Tears Left To Cry,0.037500,0.703,205947,0.696,0.000006,0,0.2740,-5.482,1,0.0529,121.969,4,0.3660,,5SxkdsY1ufZzoq9iXceLw9
2,2,The Middle,Zedd,Pop,The Middle,0.208000,0.748,184732,0.652,0.000000,7,0.1280,-3.020,1,0.0580,106.914,4,0.4310,,2ARqIya5NAuvFVHSN3bL0m
3,3,Never Be the Same (Radio Edit),Camila Cabello,Pop,Never Be the Same - Radio Edit,0.156000,0.652,227077,0.682,0.000130,0,0.1210,-4.380,1,0.0788,129.942,4,0.2690,Yearning,0BlY60NrN0fFWbdW3RW40q
4,4,All Your Fault: Pt. 2,Bebe Rexha,Pop,Meant to Be (feat. Florida Georgia Line),0.047000,0.643,163870,0.783,0.000000,10,0.0830,-6.458,1,0.0856,154.084,4,0.5790,Sensual,7iDa6hUg2VgEL1o1HjmfBn
5,5,Finesse (Remix) [feat. Cardi B],Bruno Mars,Pop,Finesse (Remix) [feat. Cardi B],0.018500,0.704,217289,0.859,0.000000,5,0.0215,-4.877,0,0.0996,105.115,4,0.9260,Cool,3Vo4wInECJQuz9BIBMOu8i
6,6,Pray For Me (with Kendrick Lamar),The Weeknd,Pop,Pray For Me (with Kendrick Lamar),0.080500,0.728,211440,0.668,0.000015,2,0.1020,-5.020,1,0.1260,100.537,4,0.1620,Cool,4ofvoCdXq99vpMZO4Cfp8x
7,7,Scary Hours,Drake,Pop,God's Plan,0.028300,0.755,198960,0.448,0.000033,7,0.5140,-9.587,1,0.1070,77.179,4,0.3180,,6T8cJz5lAqGer9GUHGyelE
8,8,Him & I,G-Eazy,Pop,Him & I,0.053400,0.589,268867,0.731,0.000000,2,0.3080,-6.343,1,0.0868,87.908,4,0.1910,Cool,3YU6vJbjYUG0tiJyXf9x5V
9,9,Perception,NF,Pop,Let You Down,0.330000,0.659,212120,0.716,0.000000,5,0.1730,-5.673,0,0.1110,148.015,4,0.4370,Rowdy,6mrKP2jyIQmM0rw6fQryjr


In [74]:
set(list(mainDF['mood']))

{'',
 'Aggressive',
 'Brooding',
 'Cool',
 'Defiant',
 'Easygoing',
 'Empowering',
 'Energizing',
 'Excited',
 'Fiery',
 'Gritty',
 'Melancholy',
 'Other',
 'Peaceful',
 'Romantic',
 'Rowdy',
 'Sensual',
 'Sentimental',
 'Somber',
 'Sophisticated',
 'Stirring',
 'Tender',
 'Upbeat',
 'Urgent',
 'Yearning'}

In [75]:
mood_map = {'Peaceful': (-2,2), 'Easygoing': (-1,2), 'Tender': (-2,1), 'Romantic': (-1,1), 
        'Upbeat': (0,2), 'Empowering': (0,1),
        'Lively': (1,2), 'Excited': (2,2), 'Stirring': (1,1), 'Rowdy': (2,1),
        'Sentimental': (-2,0), 'Sophisticated': (-1,0),
        'Sensual': (0,0), 
        'Fiery': (1,0), 'Energizing': (2,0),
        'Melancholy': (-2,-1), 'Cool': (-1,-1), 'Somber': (-2,-2), 'Gritty': (-1,-2),
        'Yearning': (0,-1), 'Serious': (0,-2),
        'Urgent': (1,-1), 'Defiant': (2,-1), 'Brooding': (1,-2), 'Aggressive': (2,-2)}

In [76]:
mood_map_DF = pd.DataFrame(mood_map)

In [77]:
mood_map_DF = mood_map_DF.T

In [78]:
mood_map_DF['energy_level'] = mood_map_DF[0]
mood_map_DF['positivity_level'] = mood_map_DF[1]

In [79]:
mood_map_DF = mood_map_DF.drop([0,1],axis=1)

In [80]:
mood_map_DF = mood_map_DF.reset_index()

In [81]:
mood_map_DF['mood'] = mood_map_DF['index']
mood_map_DF

,index,energy_level,positivity_level,mood
0,Aggressive,2,-2,Aggressive
1,Brooding,1,-2,Brooding
2,Cool,-1,-1,Cool
3,Defiant,2,-1,Defiant
4,Easygoing,-1,2,Easygoing
5,Empowering,0,1,Empowering
6,Energizing,2,0,Energizing
7,Excited,2,2,Excited
8,Fiery,1,0,Fiery
9,Gritty,-1,-2,Gritty


In [82]:
uniquetrackDF

,Unnamed: 0,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0,Falling,Lina Mayer,Pop,2wYHz0GOHV49Xezd5mWTDP,Falling,0.031500,0.447,201385,0.594,0.000001,5,0.1110,-5.996,1,0.0448,164.050,4,0.5210
1,1,No Tears Left To Cry,Ariana Grande,Pop,5SxkdsY1ufZzoq9iXceLw9,No Tears Left To Cry,0.037500,0.703,205947,0.696,0.000006,0,0.2740,-5.482,1,0.0529,121.969,4,0.3660
2,2,The Middle,Zedd,Pop,2ARqIya5NAuvFVHSN3bL0m,The Middle,0.208000,0.748,184732,0.652,0.000000,7,0.1280,-3.020,1,0.0580,106.914,4,0.4310
3,3,Never Be the Same (Radio Edit),Camila Cabello,Pop,0BlY60NrN0fFWbdW3RW40q,Never Be the Same - Radio Edit,0.156000,0.652,227077,0.682,0.000130,0,0.1210,-4.380,1,0.0788,129.942,4,0.2690
4,4,All Your Fault: Pt. 2,Bebe Rexha,Pop,7iDa6hUg2VgEL1o1HjmfBn,Meant to Be (feat. Florida Georgia Line),0.047000,0.643,163870,0.783,0.000000,10,0.0830,-6.458,1,0.0856,154.084,4,0.5790
5,5,Finesse (Remix) [feat. Cardi B],Bruno Mars,Pop,3Vo4wInECJQuz9BIBMOu8i,Finesse (Remix) [feat. Cardi B],0.018500,0.704,217289,0.859,0.000000,5,0.0215,-4.877,0,0.0996,105.115,4,0.9260
6,6,Pray For Me (with Kendrick Lamar),The Weeknd,Pop,4ofvoCdXq99vpMZO4Cfp8x,Pray For Me (with Kendrick Lamar),0.080500,0.728,211440,0.668,0.000015,2,0.1020,-5.020,1,0.1260,100.537,4,0.1620
7,7,Scary Hours,Drake,Pop,6T8cJz5lAqGer9GUHGyelE,God's Plan,0.028300,0.755,198960,0.448,0.000033,7,0.5140,-9.587,1,0.1070,77.179,4,0.3180
8,8,Him & I,G-Eazy,Pop,3YU6vJbjYUG0tiJyXf9x5V,Him & I,0.053400,0.589,268867,0.731,0.000000,2,0.3080,-6.343,1,0.0868,87.908,4,0.1910
9,9,Perception,NF,Pop,6mrKP2jyIQmM0rw6fQryjr,Let You Down,0.330000,0.659,212120,0.716,0.000000,5,0.1730,-5.673,0,0.1110,148.015,4,0.4370


In [85]:
mainDF = pd.merge(mainDF, mood_map_DF, how='left', left_on=['mood'], right_on=['mood'])

In [86]:
mainDF.to_csv('dataWithLabel1000.csv')

In [87]:
mainDF.columns

Index(['Unnamed: 0', 'album', 'artist', 'genre', 'track_name', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'mood', 'track_id', 'index',
       'energy_level', 'positivity_level'],
      dtype='object')